 <div style="clear: both; display: table;">
  <div style="border: none; float: left; width: 60%; padding: 5px">
  <h1 id="subtitle">Chapter 7: Cython</h1>
  <h2 id="subtitle">Guillermo Avendaño Franco <br> Aldo Humberto Romero</h2>
  <br>
  <img src="fig/1-line logotype124-295.png" alt="Scientific Computing with Python" style="width:50%" align="left">
  </div>
  <div style="border: none; float: left; width: 30%; padding: 5px">
  <img src="fig/SCPython.png" alt="Scientific Computing with Python" style="width:100%">
  </div>
</div>

Adapted by **Guillermo Avendaño** (WVU), **Jose Rogan** (Universidad de Chile) and **Aldo Humberto Romero** (WVU) from the [Tutorials for Stanford cs228 and cs231n](https://github.com/kuleshov/cs228-material). A large parte of the info was also built from scratch. In turn, that material was adapted by [Volodymyr Kuleshov](http://web.stanford.edu/~kuleshov/) and [Isaac Caswell](https://symsys.stanford.edu/viewing/symsysaffiliate/21335) from the `CS231n` Python tutorial by Justin Johnson (http://cs231n.github.io/python-numpy-tutorial/).

Changes to the original tutorial include strict Python 3 formats and split of the material to fit a series of lessons on Python Programming for WVU's faculty and graduate students.

The support of the National Science Foundation and the US Department of Energy under projects: DMREF-NSF 1434897, NSF OAC-1740111 and DOE DE-SC0016176 is recognized.

<div style="clear: both; display: table;">
<div style="border: none; float: left; width: 40%; padding: 10px">
<img src="fig/NSF.jpg" alt="National Science Foundation" style="width:50%" align="left">
    </div>
    <div style="border: none; float: right; width: 40%; padding: 10px">
<img src="fig/DOE.jpg" alt="National Science Foundation" style="width:50%" align="right">
</div>

## Table of Contents

Python is a great general-purpose programming language on its own. This notebook is focused on Cython. Cython extends Python syntax so that you can conveniently build C extensions, either to speed up critical code, or to integrate with C/C++ libraries. The lesson is particularly oriented to Scientific Computing. 
Other episodes in the series include:

  * Python Syntax 
  * The Python Standard Library
  * Numpy
  * Matplotlib
  * Scipy
  * Pandas
  * **Cython \[This notebook\]**

After completing all the series in this lesson you will realize that python has become a powerful environment for scientific computing at several levels, from intereactive computing to scripting to big project developments.

## Setup

In [1]:
%load_ext watermark

In [2]:
%watermark

2019-08-08T19:14:40-04:00

CPython 3.5.2
IPython 7.7.0

compiler   : GCC 5.4.0 20160609
system     : Linux
release    : 4.15.0-55-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [3]:
import cython
%watermark -iv

cython 0.29.13



# Cython: Accelerate Python Execution

Cython is a language that extends Python. With minimal changes to a python code, you can get a compiled code that runs faster. From another perspective, Cython is like a bridge that allows you to interface C and C++ routines inside python.

We will follow a very practical route to give you basic elements that will quickly improve the performance of your code with minimal effort. Scientific computing, performance is central, but also you want to optimize the time spent developing the code. The right balance between both is the goal. Using Python is an efficient way of getting a code that works, but still work must to be done to make it efficient.

Using Jupyter notebooks is really easy to "cythonize" python code.  For this you need to load the Cython magic:

In [4]:
%load_ext cython

## Original Python Code: Sieve of Eratosthenes

Let us consider a classical algorithm, the Sieve of Eratosthenes. Eratosthnes is best known for being the first person to calculate the circumference of the Earth. He is also known for inventing a clever method of identifying prime numbers. This algorithm is more than 2000 years old and it is still a reasonable first approach to find all the primes up to a given limit.

The algorithm consist in marking all the numbers as prime candidates, starting from 2 you remove the mark for all multiples of 2. The next after 2 is 3 and still marked, so it is prime and we unmark all its multiples, the next number 4 is already unmarked, so we move to the next one, 5 and unmark all its multiples. Doing that over and over we end up unmarking all the non-primes and leaving primes numbers marked.

Below is a python code implemented with pure python variables (integers and lists). 
The code happens to be very poor in performance as we are using an interpreted language and lists and loops. 
These elements make the code run very slowly. This implementation was done on purpose because it allow us to explore how to improve it from there.

In [5]:
def SieveOfEratosthenes_orig(n):

    if n%2 == 0:
        imax=int(n/2)
    else:
        imax=int((n+1)/2)

    neff= 2*(imax-1)+1
    prime = imax*[True]

    print(" Dimension of array: %d\n" % imax)
    print(" Array with indices between 0 to %d\n" % (imax-1))
    print(" Stores primality of odd numbers in range [1, %d]\n\n" % neff)

    # The loop runs over all odd numbers starting at 3
    p=3
    while (p<n/2+1):
        # If prime[p] is true, then it is a prime
        if(prime[int((p-1)/2)] is True):
            # Update all multiples of p
            i = 2*p
            while (i <= neff):
                if (i%2!=0):
                    prime[int((i-1)/2)] = False
                i += p
        p+=2

    counter = 1
    nprimes = 0
    # Print all prime numbers if the number is small otherwise just count the number of primes found
    if (imax < 10000):
        print("%10d " % 2, end='')
        # Starting with index 1 instead of zero because (2*0)+1 = 1 is not prime
        for i in range(1,imax):
            if(prime[i] == 1):
                print("%10d " % (2*i+1), end='')
                counter+=1

            if (counter == 15):
                print("")
                counter=0

        print("\n")

    else:
        for i in range(1,imax):
            if(prime[i] is True):
                nprimes+=1
        print(" Total number of primes found: %d" % nprimes)

    return nprimes

### Timing the execution

With IPython we can use the timing function from the `%timeit` magic, the code is executed several times and the best time is reported

In [6]:
t1=%timeit -o SieveOfEratosthenes_orig(50E6)

 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number

## Using Cython with pristine code

Our first movement will be to run the code unmodified, the only change is the first line in the Jupiter box indicating that we want the code to be compiled with Cython

In [7]:
%%cython -3 -a
def SieveOfEratosthenes_cython_plain(n):

    if n%2 == 0:
        imax=int(n/2)
    else:
        imax=int((n+1)/2)

    neff= 2*(imax-1)+1
    prime = imax*[True]

    print(" Dimension of array: %d\n" % imax)
    print(" Array with indices between 0 to %d\n" % (imax-1))
    print(" Stores primality of odd numbers in range [1, %d]\n\n" % neff)

    # The loop runs over all odd numbers starting at 3
    p=3
    while (p<n/2+1):
        # If prime[p] is true, then it is a prime
        if(prime[int((p-1)/2)] is True):
            # Update all multiples of p
            i = 2*p
            while (i <= neff):
                if (i%2!=0):
                    prime[int((i-1)/2)] = False
                i += p
        p+=2

    counter = 1
    nprimes = 0
    # Print all prime numbers if the number is small otherwise just count the number of primes found
    if (imax < 10000):
        print("%10d " % 2, end='')
        # Starting with index 1 instead of zero because (2*0)+1 = 1 is not prime
        for i in range(1,imax):
            if(prime[i] == 1):
                print("%10d " % (2*i+1), end='')
                counter+=1

            if (counter == 15):
                print("")
                counter=0

        print("\n")

    else:
        for i in range(1,imax):
            if(prime[i] is True):
                nprimes+=1
        print(" Total number of primes found: %d" % nprimes)

    return nprimes

With `%%cython -3 -a` the code is cythonized, ie converted into C code that is afterwards compiled and converted into a Python module. The argument `-3` is to indicate that we are using Python 3.x and `-a` to annotate the code as shown above, white lines are reagions of Python code that could be converted into pure C code. The lines are colored more and more yellow as the code on them could be less and less translated into C code. Brigth yellow meaning that those lines could not be translated and will remain as Python code.

### Timing the execution

In [8]:
t2=%timeit -o SieveOfEratosthenes_cython_plain(50E6)

 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number

## Static typing

Python is a dynamic language, meaning that you can declare variables just by initializing them with values, and you can change values for the same variable, for example declaring a variable to be a number and later on changing the variable to be a string.

This can make Python a very relaxed and comfortable language for rapid development, but that flexibility comes with a price. The Python interpreter has to do extra work to figure out which kind of varible you have associated to a given variable when you execute an operation with it. The interpreter has to search namespaces, fetch attributes and parse argument and keyword tuples. 
This is a big reason why Python is considered slow when compared with other languages like C or Fortran.

However with Cython it is possible to gain significant speed-ups through the use of ‘early binding’ programming techniques.

As you saw above, declaring the type of variable is not a necessity. Cython is able to give you some performance just from the code as it was originally. 

Providing static typing to parameters and variables is convenience to speed up your code, this is our next step. 

You should optimize where and when needed. Not doing it right and you can actually slow down your code in the case where the typing does not allow optimizations but where Cython still needs to check that the type of some object matches the declared type.

In our case, the decision is quite simple. All the variables `imax, neff, p, i, counter` and `nprimes` are integers and will be integers during the entire execution. We can declare that with the line

    cdef int imax, neff, p, i, counter, nprimes
    
That is our next step the code is the same except for that single line

In [9]:
%%cython -3 -a
def SieveOfEratosthenes_cython_static(n):

    # Type declared variables for cython
    cdef int imax, neff, p, i, counter, nprimes
    
    if n%2 == 0:
        imax=int(n/2)
    else:
        imax=int((n+1)/2)

    neff= 2*(imax-1)+1
    prime = imax*[True]

    print(" Dimension of array: %d\n" % imax)
    print(" Array with indices between 0 to %d\n" % (imax-1))
    print(" Stores primality of odd numbers in range [1, %d]\n\n" % neff)

    # The loop runs over all odd numbers starting at 3
    p=3
    while (p<n/2+1):
        # If prime[p] is true, then it is a prime
        if(prime[int((p-1)/2)] is True):
            # Update all multiples of p
            i = 2*p
            while (i <= neff):
                if (i%2!=0):
                    prime[int((i-1)/2)] = False
                i += p
        p+=2

    counter = 1
    nprimes = 0
    # Print all prime numbers if the number is small otherwise just count the number of primes found
    if (imax < 10000):
        print("%10d " % 2, end='')
        # Starting with index 1 instead of zero because (2*0)+1 = 1 is not prime
        for i in range(1,imax):
            if(prime[i] == 1):
                print("%10d " % (2*i+1), end='')
                counter+=1

            if (counter == 15):
                print("")
                counter=0

        print("\n")

    else:
        for i in range(1,imax):
            if(prime[i] is True):
                nprimes+=1
        print(" Total number of primes found: %d" % nprimes)

    return nprimes

### Timing the execution

In [10]:
t3=%timeit -o SieveOfEratosthenes_cython_static(50E6)

 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number

## Working with C arrays and pointers

One of the reasons why the Python code that we proposed initially runs so slow is that we are storing the primality condition for all odd numbers as a big list. Python list are very flexible, they are mutable, meaning that you can add and remove elements to the list and change the values on the fly. Those features are nice but also comes with a performance tag. This time we need to introduce more changes.

Instead of a list we will declare `prime` to be a pointer, basically and array of integer numbers. 
We know that we want `imax` integers to allocate. The following line will create the array.

    cdef int *prime = <int *> malloc(imax * sizeof(int))

In [11]:
%%cython -3 -a
from libc.stdio cimport printf
from libc.stdlib cimport malloc, free

def SieveOfEratosthenes_cython_carray(n):

    cdef int imax, neff, p, i, counter, nprimes

    if n%2 == 0:
        imax=int(n/2)
    else:
        imax=int((n+1)/2)

    neff= 2*(imax-1)+1

    cdef int *prime = <int *> malloc(imax * sizeof(int))
    for i in range(imax):
        prime[i]=1    

    print(" Dimension of array: %d\n" % imax)
    print(" Array with indices between 0 to %d\n" % (imax-1))
    print(" Stores primality of odd numbers in range [1, %d]\n\n" % neff)

    # The loop runs over all odd numbers starting at 3
    p=3
    while (p<n/2+1):
        # If prime[p] is true, then it is a prime
        if(prime[int((p-1)/2)] == 1):
            # Update all multiples of p
            i = 2*p
            while (i <= neff):
                if (i%2!=0):
                    prime[int((i-1)/2)] = 0
                i += p
        p+=2

    counter = 1
    nprimes = 0
    # Print all prime numbers if the number is small otherwise just count the number of primes found
    if (imax < 10000):
        print("%10d " % 2, end='')
        # Starting with index 1 instead of zero because (2*0)+1 = 1 is not prime
        for i in range(1,imax):
            if(prime[i] == 1):
                print("%10d " % (2*i+1), end='')
                counter+=1

            if (counter == 15):
                print("")
                counter=0

        print("\n")

    else:
        for i in range(1,imax):
            if(prime[i] == 1):
                nprimes+=1
        print(" Total number of primes found: %d" % nprimes)

    free(prime)
    return nprimes

### Timing the execution

In [12]:
t4=%timeit -o SieveOfEratosthenes_cython_carray(50E6)

 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number

## Python functions from Cython

Up to now, we have gained a lot of performance with just 2 extra lines of code. Most of the annotated code is mostly white, but there is still a few yellow lines. The argument to the function `n` has not being typed, so all references to it cannot be fully converted into C code. As that is the argument of the function we have to work into C functions.

There are two kinds of function definition in Cython: Python functions and C functions.
Python functions are defined using the `def` statement, the usual statement in Python. 
They take Python objects as parameters and return Python objects.

C functions are defined using the new `cdef` statement. 
They take either Python objects or C values as parameters, and can return either Python objects or C values.

Within a Cython module, Python functions and C functions can call each other freely, but only Python functions can be called from outside the module by interpreted Python code. So, any functions that you want to “export” from your Cython module must be declared as Python functions using `def`. There is also a hybrid function, called `cpdef`. A `cpdef` can be called from anywhere, but uses the faster C calling conventions when being called from other Cython code. A `cpdef` can also be overridden by a Python method on a subclass or an instance attribute, even when called from Cython. If this happens, most performance gains are of course lost and even if it does not, there is a tiny overhead in calling a `cpdef` method from Cython compared to calling a `cdef` method.

Our next step is replacing the function from Python definition to the hybrid `cpdef` definition

    cpdef int SieveOfEratosthenes_cython_cfunc(int n):

In [13]:
%%cython -3 -a
from libc.stdio cimport printf
from libc.stdlib cimport malloc, free

cpdef int SieveOfEratosthenes_cython_cfunc(int n):

    cdef int imax, neff, p, i, counter, nprimes

    if n%2 == 0:
        imax=int(n/2)
    else:
        imax=int((n+1)/2)

    neff= 2*(imax-1)+1

    cdef int *prime = <int *> malloc(imax * sizeof(int))
    for i in range(imax):
        prime[i]=1    

    print(" Dimension of array: %d\n" % imax)
    print(" Array with indices between 0 to %d\n" % (imax-1))
    print(" Stores primality of odd numbers in range [1, %d]\n\n" % neff)

    # The loop runs over all odd numbers starting at 3
    p=3
    while (p<n/2+1):
        # If prime[p] is true, then it is a prime
        if(prime[int((p-1)/2)] == 1):
            # Update all multiples of p
            i = 2*p
            while (i <= neff):
                if (i%2!=0):
                    prime[int((i-1)/2)] = 0
                i += p
        p+=2

    counter = 1
    nprimes = 0
    # Print all prime numbers if the number is small otherwise just count the number of primes found
    if (imax < 10000):
        print("%10d " % 2, end='')
        # Starting with index 1 instead of zero because (2*0)+1 = 1 is not prime
        for i in range(1,imax):
            if(prime[i] == 1):
                print("%10d " % (2*i+1), end='')
                counter+=1

            if (counter == 15):
                print("")
                counter=0

        print("\n")

    else:
        for i in range(1,imax):
            if(prime[i] == 1):
                nprimes+=1
        print(" Total number of primes found: %d" % nprimes)

    free(prime)
    return nprimes

### Timing the execution

In [14]:
t5=%timeit -o SieveOfEratosthenes_cython_cfunc(50000000)

 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number of primes found: 3001133
 Dimension of array: 25000000

 Array with indices between 0 to 24999999

 Stores primality of odd numbers in range [1, 49999999]


 Total number

## Removing the print statements

As a final and very minor change, we can remove all the print functions, they are intrisically Python functions, so those are the remaining lines of Python code.

In [15]:
%%cython -3 -a
from libc.stdio cimport printf
from libc.stdlib cimport malloc, free

cpdef int SieveOfEratosthenes_cython_noprint(int n):

    cdef int imax, neff, p, i, counter, nprimes

    if n%2 == 0:
        imax=int(n/2)
    else:
        imax=int((n+1)/2)

    neff= 2*(imax-1)+1

    cdef int *prime = <int *> malloc(imax * sizeof(int))
    for i in range(imax):
        prime[i]=1    

    #print(" Dimension of array: %d\n" % imax)
    #print(" Array with indices between 0 to %d\n" % (imax-1))
    #print(" Stores primality of odd numbers in range [1, %d]\n\n" % neff)

    # The loop runs over all odd numbers starting at 3
    p=3
    while (p<n/2+1):
        # If prime[p] is true, then it is a prime
        if(prime[int((p-1)/2)] == 1):
            # Update all multiples of p
            i = 2*p
            while (i <= neff):
                if (i%2!=0):
                    prime[int((i-1)/2)] = 0
                i += p
        p+=2

    counter = 1
    nprimes = 0
    # Print all prime numbers if the number is small otherwise just count the number of primes found
    #if (imax < 10000):
    #    print("%10d " % 2, end='')
    #    # Starting with index 1 instead of zero because (2*0)+1 = 1 is not prime
    #    for i in range(1,imax):
    #        if(prime[i] == 1):
    #            print("%10d " % (2*i+1), end='')
    #            counter+=1
    #
    #        if (counter == 15):
    #            print("")
    #            counter=0
    #
    #    print("\n")
    #
    #else:
    #    for i in range(1,imax):
    #        if(prime[i] == 1):
    #            nprimes+=1
    #    print(" Total number of primes found: %d" % nprimes)

    for i in range(1,imax):
        if(prime[i] == 1):
            nprimes+=1
    
    free(prime)
    return nprimes

### Timing the execution

In [16]:
t6=%timeit -o SieveOfEratosthenes_cython_noprint(50000000)

645 ms ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Collecting the timings

In [17]:
import pandas as pd

In [18]:
timings={'original': t1.all_runs, 'cython_pristine': t2.all_runs, 'static_typing': t3.all_runs, 'c_arrays': t4.all_runs, 
         'hybrid_function': t5.all_runs, 'noprint': t6.all_runs}
cols=['original', 'cython_pristine', 'static_typing', 'c_arrays', 'hybrid_function', 'noprint']

In [19]:
df=pd.DataFrame(timings, columns=cols)

In [20]:
df

,original,cython_prisitne,static_typing,c_arrays,hybrid_function,noprint
0,38.580695,16.232022,8.943028,2.038210,0.709561,0.640438
1,38.863659,16.631265,8.869990,2.057527,0.739463,0.610267
2,38.639746,16.818921,8.521833,2.050514,0.714786,0.668699
3,38.421798,16.300895,8.720707,2.061960,0.721741,0.649594
4,38.101374,15.364543,8.421718,2.043911,0.723320,0.665941
5,38.467555,15.416203,8.454155,2.087027,0.723658,0.613113
6,38.033146,15.882903,8.302021,2.183538,0.698322,0.667784


In [21]:
df.describe()

,original,cython_prisitne,static_typing,c_arrays,hybrid_function,noprint
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
mean,38.443996,16.092393,8.604779,2.074670,0.718693,0.645119
std,0.294493,0.564517,0.242532,0.050529,0.012912,0.025114
min,38.033146,15.364543,8.302021,2.038210,0.698322,0.610267
25%,38.261586,15.649553,8.437936,2.047213,0.712173,0.626776
50%,38.467555,16.232022,8.521833,2.057527,0.721741,0.649594
75%,38.610221,16.466080,8.795349,2.074494,0.723489,0.666862
max,38.863659,16.818921,8.943028,2.183538,0.739463,0.668699
